# Synth. Obs.: Analytic spiral

We create synthetic observations for the Magritte model of the analytic spiral that was created in the [this example](../0_creating_models/1_create_analytic_spiral.ipynb).

## Setup

Import the required functionalty.

In [1]:
import magrittetorch.model.model     as magritte   # Core functionality
import magrittetorch.algorithms.solvers as solvers       # Plotting
import magrittetorch.tools.plot    as plot      # Save fits
import os
import torch

from astropy import units, constants              # Unit conversions

Define a working directory (you will have to change this). We assume here that the scripts of the [this example](../0_creating_models/1_create_analytic_spiral.ipynb) have already been executed and go back to that working directory.

In [2]:
wdir = "/lhome/thomasc/Magrittetorch-examples/Analytic_spiral/"

Define file names.

In [3]:
model_file = os.path.join(wdir, 'model_analytic_spiral.hdf5')   # Analytic spiral Magritte model

Load the Magritte model.

In [4]:
model = magritte.Model(model_file)
model.read(legacy_mode=False) # Load Magrittetorch model

Reading model from:  /lhome/thomasc/Magrittetorch-examples/Analytic_spiral/model_analytic_spiral.hdf5
Reading Magrittetorch model


## Model the medium

Initialize the model.

In [5]:
model.dataCollection.infer_data() # Correctly initialize all data

In this example we will work with the LTE level populations and **do not demand** statistical equilibrium.

In [6]:
# If you have a GPU, we can use it to speed up the computations
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Getting the full model on GPU might be a bit too much for its memory, so you might want to use your CPU instead for the computations
# device = torch.device("cpu")

# Iterate level populations until statistical equilibrium
# solvers.compute_level_populations(model, device=device, 20, True)

## Make synthetic observations

Now we can make synthetic observations of the model.

In [7]:
fcen = model.sources.lines.lineProducingSpecies[0].linedata.frequency.get()[0]
vpix = 1300 # velocity pixel size [m/s] 
dd   = vpix * 25 / constants.c.si.value
fmin = fcen - fcen*dd
fmax = fcen + fcen*dd

# Make sure that all input tensors lie on the same device (CPU or GPU)
device = torch.device("cpu")
freqs = torch.linspace(fmin, fmax, 51, dtype=torch.float64, device=device)
# Ray orthogonal to image plane
ray_nr = 3
raydir = model.geometry.rays.direction.get(device)[ray_nr]
solvers.image_model(model, raydir, freqs, torch.device("cpu"), Nxpix=256, Nypix=256)#using a resolution of 256x256 for the image. 

## Plot observations

Plot the resulting channel maps with matplotlib.

In [8]:
plot.image_mpl(
    model,
    image_nr =  -1,
    zoom     = 1.3,
    npix_x   = 256,
    npix_y   = 256,
    x_unit   = units.au,
    v_unit   = units.km / units.s)

100%|████████████████████████████████████████████████████| 51/51 [00:29<00:00,  1.71it/s]


interactive(children=(IntSlider(value=25, description='v', max=50), Output()), _dom_classes=('widget-interact'…

<function magrittetorch.tools.plot.image_mpl.<locals>.<lambda>(v)>

(The plot is only interactive in a live notebook.)

Save the image cube in a fits file.

In [9]:
plot.save_fits(model)

Written file to: /lhome/thomasc/Magrittetorch-examples/Analytic_spiral/images/image.fits


(Optional: To create your own plots) Overview of data stored in the Image object

In [10]:
xdir = model.images[-1].image_direction_x.get_astropy()#directions of the x-and y-vectors of the image
ydir = model.images[-1].image_direction_y.get_astropy()
zdir = model.images[-1].image_direction_z.get_astropy()#this is direction in which we observe the object
print("image directions: ", xdir, ydir, zdir)
nfreqs = model.images[-1].nfreqs.get() #number of frequency bins
freqs = model.images[-1].freqs.get_astropy() #frequency bins [Hz]
print("# of frequencies: ", nfreqs, " frequencies :", freqs)
ImX = model.images[-1].imX.get_astropy()#X position in image [m]
ImY = model.images[-1].imY.get_astropy()#Y position in image [m]
I = model.images[-1].I.get_astropy()#Intensity at the corresponding ImX, ImY position [W/(m^2*Hz*sr)], at a given frequency bin
# print("Intensities :", I, " ImX:", ImX, "ImY:", ImY) #prints a lot of output

image directions:  [ 0.61267884 -0.79033198  0.        ] [-0.65352411 -0.50662305 -0.5623516 ] [ 0.44444445  0.34454093 -0.82689823]
# of frequencies:  51  frequencies : [1.15258709e+11 1.15259208e+11 1.15259708e+11 1.15260208e+11
 1.15260708e+11 1.15261208e+11 1.15261708e+11 1.15262207e+11
 1.15262707e+11 1.15263207e+11 1.15263707e+11 1.15264207e+11
 1.15264707e+11 1.15265207e+11 1.15265706e+11 1.15266206e+11
 1.15266706e+11 1.15267206e+11 1.15267706e+11 1.15268206e+11
 1.15268706e+11 1.15269205e+11 1.15269705e+11 1.15270205e+11
 1.15270705e+11 1.15271205e+11 1.15271705e+11 1.15272205e+11
 1.15272704e+11 1.15273204e+11 1.15273704e+11 1.15274204e+11
 1.15274704e+11 1.15275204e+11 1.15275704e+11 1.15276203e+11
 1.15276703e+11 1.15277203e+11 1.15277703e+11 1.15278203e+11
 1.15278703e+11 1.15279203e+11 1.15279702e+11 1.15280202e+11
 1.15280702e+11 1.15281202e+11 1.15281702e+11 1.15282202e+11
 1.15282702e+11 1.15283201e+11 1.15283701e+11] Hz
